# MAPPO Integration

This runs the integrated MAPPO algorithm with the patrolling zoo.

In [10]:
%reload_ext autoreload
%autoreload 2
from onpolicy.scripts.train.train_patrolling import get_config, parse_args, main

import os
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [11]:
parser = get_config()
all_args = parse_args([], parser)

all_args.experiment_name = "asyncActions"
all_args.env_name = "Patrolling"
all_args.user_name = "anthony-goeckner"

all_args.num_agents = 2
all_args.agent_speed = 10.0
all_args.observe_method = "ajg_new"
all_args.async_actions = True

all_args.graph_name = "4nodes"
all_args.graph_file = f"patrolling_zoo/env/{all_args.graph_name}.graph"
all_args.num_env_steps = 10e5 #total number of steps
all_args.episode_length = 200 #number of steps in a training episode
all_args.max_cycles = all_args.episode_length #number of steps in an environment episode

all_args.algorithm_name = "mappo"
all_args.use_recurrent_policy = True
all_args.use_naive_recurrent_policy = False
all_args.use_centralized_V = True
all_args.use_gae = False
all_args.use_gae_amadm = True
all_args.share_policy = False

all_args.n_rollout_threads = 1
all_args.cuda = True
all_args.cuda_idx = 0

all_args.use_wandb = False

In [12]:
main([], parsed_args = all_args)

u are choosing to use mappo, we set use_recurrent_policy & use_naive_recurrent_policy to be False
choose to use gpu...

 Env Patrolling Algo mappo Exp asyncActions updates 0/5000 episodes, total num timesteps 200/1000000.0, FPS 87.

average episode rewards is 148800.0

 Env Patrolling Algo mappo Exp asyncActions updates 1/5000 episodes, total num timesteps 400/1000000.0, FPS 86.

average episode rewards is 85166.66259765625

 Env Patrolling Algo mappo Exp asyncActions updates 2/5000 episodes, total num timesteps 600/1000000.0, FPS 86.

average episode rewards is 222666.6748046875

 Env Patrolling Algo mappo Exp asyncActions updates 3/5000 episodes, total num timesteps 800/1000000.0, FPS 87.

average episode rewards is 170909.09423828125

 Env Patrolling Algo mappo Exp asyncActions updates 4/5000 episodes, total num timesteps 1000/1000000.0, FPS 88.

average episode rewards is 125600.0

 Env Patrolling Algo mappo Exp asyncActions updates 5/5000 episodes, total num timesteps 1200/1000000